In [13]:
import duckdb
import pandas as pd

con = duckdb.connect(database=':memory:')

## 1. 테이블 생성 (CREATE TABLE)
print("1. 'employees_temp' 테이블 생성")
print("---")
con.execute("""
CREATE TABLE employees_temp (
    employee_id INTEGER,
    name VARCHAR,
    department VARCHAR,
    salary INTEGER,
    is_active BOOLEAN,
    hire_year INTEGER
);
""")

1. 'employees_temp' 테이블 생성
---


In [14]:

## 2. 데이터 삽입 (INSERT INTO)
con.execute("""
INSERT INTO employees_temp VALUES
(101, 'Alice', 'HR', 60000, TRUE, 2020),
(102, 'Bob', 'IT', 85000, TRUE, 2018),
(103, 'Charlie', 'Sales', 55000, TRUE, 2022),
(104, 'David', 'IT', 70000, FALSE, 2023),
(105, 'Eve', 'HR', 72000, TRUE, 2019),
(106, 'Frank', 'Sales', 58000, FALSE, 2024);
""")

print("초기 데이터 상태:")
print(con.sql("SELECT * FROM employees_temp;").df())


초기 데이터 상태:
   employee_id     name department  salary  is_active  hire_year
0          101    Alice         HR   60000       True       2020
1          102      Bob         IT   85000       True       2018
2          103  Charlie      Sales   55000       True       2022
3          104    David         IT   70000      False       2023
4          105      Eve         HR   72000       True       2019
5          106    Frank      Sales   58000      False       2024


In [15]:

## 3. 데이터 조회 (SELECT) - 다양한 WHERE 조건 사용

# 조건 1: 급여가 70000 이상인 직원
print("급여가 70000 이상인 직원:")
print(con.sql("SELECT name, department, salary FROM employees_temp WHERE salary >= 70000;").df())


급여가 70000 이상인 직원:
    name department  salary
0    Bob         IT   85000
1  David         IT   70000
2    Eve         HR   72000


In [16]:

# 조건 2: IT 부서이면서 활성 상태(TRUE)인 직원 (AND 사용)
print("IT 부서이면서 활성 상태인 직원:")
print(con.sql("SELECT name, department, is_active FROM employees_temp WHERE department = 'IT' AND is_active = TRUE;").df())


IT 부서이면서 활성 상태인 직원:
  name department  is_active
0  Bob         IT       True


In [17]:

# 조건 3: Sales 또는 HR 부서 직원 (OR 사용)
print("Sales 또는 HR 부서 직원:")
print(con.sql("SELECT name, department FROM employees_temp WHERE department = 'Sales' OR department = 'HR';").df())


Sales 또는 HR 부서 직원:
      name department
0  Charlie      Sales
1    Frank      Sales
2    Alice         HR
3      Eve         HR


In [18]:

# 조건 4: 2020년 이전에 입사한 직원 (hire_year < 2020)
print("2020년 이전에 입사한 직원:")
print(con.sql("SELECT name, hire_year FROM employees_temp WHERE hire_year < 2020;").df())


2020년 이전에 입사한 직원:
  name  hire_year
0  Bob       2018
1  Eve       2019


In [19]:

# 조건 5: 이름이 'D'로 시작하는 직원 (LIKE 사용)
print("이름이 'D'로 시작하는 직원:")
print(con.sql("SELECT name FROM employees_temp WHERE name LIKE 'D%';").df())
print("="*50)


이름이 'D'로 시작하는 직원:
    name
0  David


In [21]:

## 4. 데이터 갱신 (UPDATE) - 다양한 WHERE 조건 사용

# 갱신 1: IT 부서 직원들의 급여를 10% 인상
print("IT 부서 직원 급여 10% 인상")
con.execute("UPDATE employees_temp SET salary = salary * 1.10 WHERE department = 'IT';")

print(con.sql("SELECT * FROM employees_temp;").df())

IT 부서 직원 급여 10% 인상
   employee_id     name department  salary  is_active  hire_year
0          101    Alice         HR   60000       True       2020
1          102      Bob         IT  102850       True       2018
2          103  Charlie      Sales   55000       True       2022
3          104    David         IT   84700      False       2023
4          105      Eve         HR   72000       True       2019
5          106    Frank      Sales   58000      False       2024


In [22]:

# 갱신 2: 2023년 이후 입사한 비활성(FALSE) 직원을 활성(TRUE)으로 변경
print("2023년 이후 입사한 비활성 직원을 활성으로 변경 (데이터 없음, 실습용)")
con.execute("UPDATE employees_temp SET is_active = TRUE WHERE hire_year >= 2023 AND is_active = FALSE;")

print(con.sql("SELECT * FROM employees_temp;").df())

2023년 이후 입사한 비활성 직원을 활성으로 변경 (데이터 없음, 실습용)
   employee_id     name department  salary  is_active  hire_year
0          101    Alice         HR   60000       True       2020
1          102      Bob         IT  102850       True       2018
2          103  Charlie      Sales   55000       True       2022
3          104    David         IT   84700       True       2023
4          105      Eve         HR   72000       True       2019
5          106    Frank      Sales   58000       True       2024


In [23]:

## 5. 데이터 삭제 (DELETE) - 다양한 WHERE 조건 사용

# 삭제 1: 급여가 75000 이상인 직원 삭제 (Bob과 Eve가 해당)
con.execute("DELETE FROM employees_temp WHERE salary >= 75000;")
print("삭제 후 상태 (4개 행 남음: Alice, Charlie, David, Frank)")
print(con.sql("SELECT * FROM employees_temp;").df())


삭제 후 상태 (4개 행 남음: Alice, Charlie, David, Frank)
   employee_id     name department  salary  is_active  hire_year
0          101    Alice         HR   60000       True       2020
1          103  Charlie      Sales   55000       True       2022
2          105      Eve         HR   72000       True       2019
3          106    Frank      Sales   58000       True       2024


In [24]:

# 삭제 2: HR 또는 Sales 부서 직원 삭제 (Alice, Charlie, Frank가 해당)
con.execute("DELETE FROM employees_temp WHERE department = 'HR' OR department = 'Sales';")
print("삭제 후 상태 (1개 행 남음: David)")
print(con.sql("SELECT * FROM employees_temp;").df())


삭제 후 상태 (1개 행 남음: David)
Empty DataFrame
Columns: [employee_id, name, department, salary, is_active, hire_year]
Index: []


In [ ]:

# 연결 종료
con.close()
print("데이터베이스 연결 종료.")